In [1]:
from data_utils import load_data, preprocess_data_faiss, preprocess_data_faiss_test

In [2]:
data = load_data("data")

In [5]:
embeddings = preprocess_data_faiss(data)
embeddings.shape

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["body"] = posts["body"].map(clean_html)


(11812, 2048)

In [4]:
posts, test_data = preprocess_data_faiss_test(data)
test_data

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["id"] = posts["id"].astype("int32")


,postid,relatedpostid,linktypeid
0,9474,7445,0
1,3348,6112,0
2,7821,6298,1
3,4331,2804,1
4,1448,858,1
...,...,...,...
8539,10885,5852,0
8540,7045,2171,0
8541,7093,163,1
8542,657,408,0


In [6]:
import faiss
import numpy as np

In [8]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [20]:
test_embeddings = embeddings[test_data["postid"]]

In [25]:
D, I = index.search(test_embeddings, k=6)

In [29]:
for k in range(6, 22, 5):
    print(f"TOP {k - 1} RECOMMENDATIONS")

    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Accuracy: {correct / test_data.shape[0]}\n")

TOP 5 RECOMMENDATIONS
Accuracy: 0.8921016397546626

TOP 10 RECOMMENDATIONS
Accuracy: 0.9454249593190637

TOP 15 RECOMMENDATIONS
Accuracy: 0.9644511202903993

TOP 20 RECOMMENDATIONS
Accuracy: 0.973964200776067



In [30]:
random_embedding = embeddings[
    np.random.choice(embeddings.shape[0])
].reshape(1, -1)

In [31]:
D, I = index.search(random_embedding, k=6)

In [32]:
print(f"Distances: {D}")
print(f"Indices: {I}")

Distances: [[0.        1.3095431 1.4885429 1.5132227 1.5407709 1.5641645]]
Indices: [[5547 9921 9412 5556 2301 5555]]


In [34]:
print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {posts['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {posts['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {posts['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {posts['body'].iloc[I[0][i]]}"
    )

===ORIGINAL POST===
TITLE: What are the core differences between strawberry varieties?
CONTENT: What are the core differences between strawberry varieties?  (For commercially available strawberries in North America and worldwide0)  For example, strawberries from Oregon and Washington are smaller and redder than those from California, but they don't have as long a shelf life.  
(Also, what names - rather than their origin - identify these different strawberries?)

===RECOMMENDATION 1 (distance = 1.3095431327819824)===
TITLE: Have strawberries changed or have I?
CONTENT: Strawberries used to be one of my favorite fruits. I am continually disappointed when I bite into them lately and they have little flavor unless it's that slightly fermented taste. I am sure that large fruit distributors supply all the local grocers with bright-red, bruise-resistant product without any concern for flavor. However, even when I go picking in the short northeastern season, it's hit or miss. It's almost pick